In [231]:
import pandas as pd
pd.options.mode.chained_assignment = None

import math
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding

In [232]:
# Combine both datasets into one dataframe
data = pd.read_json('Sarcasm_Headlines_Dataset.json', lines=True)
data = pd.merge(data, pd.read_json('Sarcasm_Headlines_Dataset_v2.json', lines=True))
data.shape[0]

26712

# Data cleaning

In [233]:
# Split into test, validation, and train
index = math.floor(data.shape[0] * 0.8)

train_df = data[0:index]
test_df = data[index:]

In [234]:
train_df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [235]:
# Remove article link
train_df.drop(labels='article_link', axis=1, inplace=True)
test_df.drop(labels='article_link', axis=1, inplace=True)
train_df.shape[0]

21369

In [236]:
# Drop duplicate rows
train_df.drop_duplicates(inplace=True)
train_df.shape[0]

21289

In [237]:
# Drop rows with NA values
train_df.dropna(inplace=True)
train_df.shape[0]

21289

In [238]:
# Ensure the is_sarcastic column only contains 1s and 0s
train_df[(train_df.is_sarcastic <= 1)]
train_df[(train_df.is_sarcastic >= 0)]
train_df.shape[0]

21289

# Sample model

I used this link for inspiration on the model design: https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/ although I couldn't quite get it to work (I'm assuming the issue has something to do with the shape of my dataframe).

In [239]:
# This type of model requires a fixed width column, so ensure all columns are same width
train_df.headline.str.len().max()

254

In [251]:
# train_df['headline'] = train_df['headline'].str.pad(250, side='right', fillchar=' ')
train_df['headline'] = train_df.headline.astype(str).str.pad(260, side='right', fillchar='0')
x_train = np.reshape(train_df['headline'].values, (-1,1))
y_train = np.reshape(train_df['is_sarcastic'].values, (-1,1))

In [241]:
train_df.shape[0]

21289

In [246]:
model = Sequential()
model.add(Embedding(train_df.shape[0], 64, input_length=None))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, None, 64)          1362496   
                                                                 
 lstm_11 (LSTM)              (None, 100)               66000     
                                                                 
 dense_11 (Dense)            (None, 1)                 101       
                                                                 
Total params: 1,428,597
Trainable params: 1,428,597
Non-trainable params: 0
_________________________________________________________________
None


In [247]:
train_df['headline'].shape

(21289,)

In [248]:
train_df['is_sarcastic'].shape

(21289,)

In [254]:
model.fit(train_df['headline'], train_df['is_sarcastic'], 
          validation_data=(test_df['headline'], test_df['is_sarcastic']), epochs=2, batch_size=64)

Epoch 1/2


UnimplementedError: ignored

In [ ]:
results = model.evaluate(test_df['headline'], test_df['is_sarcastic'], verbose=0)
print("Accuracy: " + (scores[1]*100))